In [1]:
import os
import re
from collections import Counter
os.system("pip install gensim pandas")

Defaulting to user installation because normal site-packages is not writeable
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.0/61.0 kB 2.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.4/60.4 kB 2.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 159.4/159.4 kB 3.8 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 26.5/26.5 MB 46.8 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.6/11.6 MB 60.5 MB/s eta 0:00:00:00:010:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.1/12.1 MB 60.8 MB/s eta 0:00:00:00:010:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.0/13.0 MB 51.3 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 310.7/310.7 kB 4.6 MB/s eta 0:00:00:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.6/4.6 MB 39.3 MB/s eta 0:00:00:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.2/302.2 kB 3.7 MB/s eta 0:00:00:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 16.2


[notice] A new release of pip is available: 23.3.2 -> 24.0
[notice] To update, run: pip install --upgrade pip


0

In [2]:
import gensim
import gensim.downloader
import gensim.downloader as api

import pandas as pd
import matplotlib.pyplot as plt
from sklearn.decomposition import TruncatedSVD
from sklearn.decomposition import PCA

In [3]:
model_name = "english_word2vec.bin"
model_path = os.path.join("..","in","model",model_name)
model = gensim.models.KeyedVectors.load(model_path)

In [4]:
#data
data_name = "data_spoty.csv"
data_path = os.path.join("..","in","data",data_name)
data = pd.read_csv(data_path)

In [5]:
#subset
singer = "ABBA"
sub_data = data.loc[data['artist'] == singer]

In [6]:
#embed
query_word = "love"
n_similar = 12
query_array = model.most_similar(query_word,topn=n_similar)
query_array.append((query_word, 1))

In [69]:
# find in subset (subset rows by indexing row number to sub_data.loc)
max_row = len(sub_data.index)
occurence_count_full = {"song":[]}
for row_index in range(0,max_row):
    row = sub_data.loc[row_index]
    song_title = row["song"]
    song_lyric = row["text"]
    artist = row["artist"]
    #clean lyrics to be compatible with the word embedding labels
    clean_lyric = re.sub(r"""
           [,.;@#?!&$\n]+  # Accept one or more copies of punctuation
           \ *           # plus zero or more copies of a space,
           """,
           " ",          # and replace it with a single space
           song_lyric.lower(), flags=re.VERBOSE)
    
    #for each word in list
    occurence_count = {}
    for word_value_pair in query_array:
        target_word = word_value_pair[0]
        #count list elements in lyric
        n_occurences = clean_lyric.split().count(target_word)
        temporary_dict = {artist:song_title,target_word:n_occurences}
        #check counts in the dictionary. if none found, it isn't saved
        non_zero_counter = 0
        for count in temporary_dict.values():
            if type(count) != str: #the first is the title so that is a str

                if count > 0:
                    non_zero_counter += 1
                else:
                    pass

            else:
                pass

        if non_zero_counter > 0:
            occurence_count.update(temporary_dict)
        else:
            pass

    #append dict if not empty
    if len(occurence_count) > 0:     
        occurence_count_full["song"].append(occurence_count)
    else:
        pass


    


In [149]:
#extended table
ex_data = pd.DataFrame(occurence_count_full["song"])
ex_data
#small info table
small_df = pd.DataFrame(columns=["Artist","Term","Percentage","All_Percentage","N_Songs"])
#add terms
small_df.Term = ex_data.columns[1:len(ex_data.columns)]
#add artist
small_df.Artist = ex_data.columns[0] 
#add n_songs
small_df.N_Songs = max_row
#add all_percentage
small_df.All_Percentage = round(len(ex_data)/max_row*100,2)
#add percentages for each query term
#look through all colums of the terms in the big set, find non zeros, make it a list
ex_data.fillna(0)
percentages = []
for col in ex_data.columns[1:len(ex_data.columns)]:
    n_z_count = non_zero_counter(ex_data[col])
    percentages.append(round(n_z_count/max_row*100,2))

small_df.Percentage = percentages


In [150]:
small_df

,Artist,Term,Percentage,All_Percentage,N_Songs
0,ABBA,love,35.40,39.82,113
1,ABBA,hate,2.65,39.82,113
2,ABBA,loving,2.65,39.82,113
3,ABBA,loved,2.65,39.82,113
4,ABBA,loves,1.77,39.82,113
5,ABBA,adore,0.88,39.82,113


In [131]:
#functions !!!!
def non_zero_counter(list_of_values):
    non_zero_counter = 0
    for count in list_of_values:
        if type(count) != str: #the first is the title so that is a str

            if count > 0:
                    non_zero_counter += 1
        else:
            pass
    else:
        pass

    return non_zero_counter

In [ ]:
#testing faculitiy

class Extended_Query:

    def __init__(self, data, model, artist, q_word, n_embeds):
        self.data = data
        self.model = model
        self.n_embeds = n_embeds
        self.artist = artist
        self.q_word = q_word
    
    def create_query(self):
        """ Method for creating a query from model and data,
        based on artist and query extension size
        - data: csv file with columns: 'artist', title as 'song', lyrics as 'text
        - model: word embedding model
        - artist: name of artist
        - q_word: lowercase str without punctuations.
        - n_embed: additional query extensions based on proximity in the embedding model"""
        #subset
        singer = self.artist
        sub_data = self.data.loc[self.data['artist'] == singer]
        #embed
        query_word = self.q_word
        n_similar = self.n_embeds
        query_array = model.most_similar(query_word,topn=n_similar)
        query_array.append((query_word, 1))

        # find in subset (subset rows by indexing row number to sub_data.loc)
        max_row = len(sub_data.index)
        occurence_count_full = {"song":[]}
        for row_index in range(0,max_row):
            row = sub_data.loc[row_index]
            song_title = row["song"]
            song_lyric = row["text"]
            artist = row["artist"]
            #clean lyrics to be compatible with the word embedding labels
            clean_lyric = clean_text(song_lyric)
            #for each word in list
            occurence_count = {}
            for word_value_pair in query_array:
                target_word = word_value_pair[0]
                #count list elements in lyric
                n_occurences = clean_lyric.split().count(target_word)
                temporary_dict = {artist:song_title,target_word:n_occurences}
                #check counts in the dictionary. if none found, it isn't saved
                n_z_counter = non_zero_counter(temporary_dict.values())

                if n_z_counter > 0:
                    occurence_count.update(temporary_dict)
                else:
                    pass

            #append dict if not empty
            if len(occurence_count) > 0:     
                occurence_count_full["song"].append(occurence_count)
            else:
                pass
    return occurence_count_full
            

In [151]:
def clean_text(text):
    clean_text = re.sub(r"""
                    [,.;@#?!&$\n]+  # Accept one or more copies of punctuation
                    \ *           # plus zero or more copies of a space,
                    """,
                    " ",          # and replace it with a single space
                    text.lower(), flags=re.VERBOSE)

In [ ]:
def make_output_tables(query_result):
    #make tables
    #extended table
    ex_data = pd.DataFrame(query_result["song"])
    ex_data
    #small info table
    small_df = pd.DataFrame(columns=["Artist","Term","Percentage","All_Percentage","N_Songs"])
    #add terms
    small_df.Term = ex_data.columns[1:len(ex_data.columns)]
    #add artist
    small_df.Artist = ex_data.columns[0] 
    #add n_songs
    small_df.N_Songs = max_row
    #add all_percentage
    small_df.All_Percentage = round(len(ex_data)/max_row*100,2)
    #add percentages for each query term
    #look through all colums of the terms in the big set, find non zeros, make it a list
    ex_data.fillna(0)
    percentages = []
    for col in ex_data.columns[1:len(ex_data.columns)]:
        n_z_count = non_zero_counter(ex_data[col])
        percentages.append(round(n_z_count/max_row*100,2))

    small_df.Percentage = percentages

    return ex_data, small_df

In [ ]:
# van-e, melyik, hányszor
# kell egy extended df meg egy quick df.
#cs (Artist,title,all_freq/all_songs,ind_count,ind_freq/all_songs)